In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from utils import get_knn_objects, get_sample_1k_objects
from enums import DatasetDirs

from LMI import LMI
from knn_search import evaluate_knn_per_query

22-03-21 13:43 INFO: Loading faiss with AVX2 support.
22-03-21 13:43 INFO: Loading faiss.


In [3]:
li = LMI(DatasetDirs.PROFI_1M.value)
df = li.get_dataset(normalize = True)

Loading labels


/auto/brno6/home/wollf/learned-indexes/learned-indexes/dataset_parsing.py:147: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_i = pd.concat([df_i, df_])


Loading descriptors


22-03-21 14:09 INFO: Loaded dataset of shape: (1000000, 4099)


In [ ]:
df_tmp = df.copy(deep = True)
df_tmp[df.columns] = df_tmp[df.columns].apply(np.float64)
df_tmp["L1"] = df_tmp["L1"].astype(np.float64)
df_tmp["L2"] = df_tmp["L2"].astype(np.float64)
df_tmp["objecht_id"] = df_tmp["objecht_id"].astype(np.float64)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 134339 to 451787
Columns: 4099 entries, 0 to object_id
dtypes: float16(4096), int64(3)
memory usage: 7.7 GB


In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,4089,4090,4091,4092,4093,4094,4095,L1,L2,object_id
134339,0.000000,0.0,0.0,0.812500,0.000000,0.000000,0.093811,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.239258,0.000000,0.000000,45,1,737487
676402,0.000000,0.0,0.0,0.000000,0.479004,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.453125,0.000000,0.0,0.000000,0.000000,0.473877,1.716797,21,16,1323491
894859,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.361084,17,65,1747783
940991,1.519531,0.0,0.0,1.664062,0.000000,0.436279,0.039001,0.097412,0.958008,0.000000,...,0.000000,3.138672,0.0,0.030518,0.081543,0.000000,2.625000,24,47,1795057
636944,0.418457,0.0,0.0,0.000000,0.412842,0.023544,0.000000,0.000000,4.195312,0.058716,...,0.000000,0.000000,0.0,0.000000,0.000000,2.152344,0.000000,51,44,1268232


In [ ]:
df_result = li.train(df, {"GMM": [{'comp': 127, 'cov_type': 'diag', 'max_iter': 1, 'init_params': 'random'},
                                  {'comp': 50, 'cov_type': 'diag', 'max_iter': 1, 'init_params': 'random'}]},
                     should_erase = True)

In [6]:
df_result

,L1_pred,L2_pred,0,1,2,3,4,5,6,7,...,4089,4090,4091,4092,4093,4094,4095,L1,L2,object_id
0,1,20,0.000000,0.000000,0.000000,1.774414,0.463867,0.000000,0.0,0.003592,...,1.776367,0.000000,0.026718,0.000000,0.546387,0.000000,0.441895,11,63,138576
1,8,76,1.047852,4.812500,0.435547,0.000000,0.032379,0.000000,0.0,0.029312,...,0.374512,0.000000,0.000000,0.000000,0.000000,0.000000,0.018524,40,59,1600526
2,4,17,0.000000,0.000000,0.000000,1.249023,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.200562,0.000000,0.000000,0.000000,0.000000,45,41,413941
3,4,69,0.015358,0.000000,0.000000,0.000000,0.354004,0.628418,0.0,0.082886,...,4.871094,0.000000,0.000000,0.000000,0.000000,0.000000,0.044250,17,18,755302
4,0,39,0.000000,0.000000,0.000000,0.019302,0.148804,5.734375,0.0,1.701172,...,0.111267,0.000000,0.363281,1.598633,1.673828,0.000000,0.373535,22,43,1696241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9,71,0.013176,0.000000,0.000000,0.000000,5.218750,0.000000,0.0,0.377686,...,0.000000,4.953125,0.000000,0.983887,0.000000,0.000000,0.000000,31,17,837948
9996,9,72,0.000000,0.000000,0.000000,0.000000,3.197266,0.417969,0.0,0.061554,...,0.000000,3.035156,0.000000,0.000000,0.000000,0.000000,1.696289,7,23,1084160
9997,7,35,0.000000,1.436523,0.000000,0.000000,0.011971,0.000000,0.0,0.000000,...,5.519531,0.000000,1.749023,0.000000,0.000000,0.000000,0.000000,28,26,1849762
9998,12,38,0.000000,0.000000,0.000000,0.000000,0.128784,9.703125,0.0,0.039337,...,0.343262,0.000000,0.000000,0.779785,0.000000,0.000000,0.244141,25,71,1756650
